In [ ]:
import pandas as pd
import os
import chardet
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import openpyxl
from openpyxl.styles import PatternFill, Font, Border, Side
import re
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk

class EmailApp:
    def __init__(self, root):
        self.root = root
        self.root.title("파트너 및 이메일 설정")
        self.dealer_info = {
            "에이치케이씨스템": "heebon.park@kt.com",
            "케이엔에스네트웍": "heebon.park@kt.com",
            "해빛씨엔에스": "bough38@naver.com",
            "비즈원플러스": "bough38@naver.com",
            "하슬라통신": "bough38@naver.com",
            "미라클 이노베이션": "bough38@naver.com",
            "성우네트웍스": "bough38@naver.com"
            "주식회사원정보통신": "bough38@naver.com",            
        }
        self.setup_ui()

    def setup_ui(self):
        frame = tk.Frame(self.root)
        frame.pack(pady=10)

        self.tree = ttk.Treeview(frame, columns=('상호', '이메일'), show='headings')
        self.tree.heading('상호', text='상호')
        self.tree.heading('이메일', text='이메일')
        self.tree.pack(side=tk.LEFT)

        scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.load_dealers()

        form_frame = tk.Frame(self.root)
        form_frame.pack(pady=10)

        tk.Label(form_frame, text="상호").grid(row=0, column=0)
        tk.Label(form_frame, text="이메일").grid(row=1, column=0)
        self.entry_name = tk.Entry(form_frame)
        self.entry_name.grid(row=0, column=1)
        self.entry_email = tk.Entry(form_frame)
        self.entry_email.grid(row=1, column=1)

        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        tk.Button(button_frame, text="추가", command=self.add_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="수정", command=self.edit_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="삭제", command=self.delete_dealer).pack(side=tk.LEFT, padx=5)
        tk.Button(button_frame, text="저장 및 닫기", command=self.save_and_close).pack(side=tk.LEFT, padx=5)

    def load_dealers(self):
        for name, email in self.dealer_info.items():
            self.tree.insert('', tk.END, values=(name, email))

    def add_dealer(self):
        name = self.entry_name.get().strip()
        email = self.entry_email.get().strip()
        if name and email:
            self.tree.insert('', tk.END, values=(name, email))
            self.dealer_info[name] = email
            self.entry_name.delete(0, tk.END)
            self.entry_email.delete(0, tk.END)

    def edit_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            old_name = item['values'][0]
            old_email = item['values'][1]
            new_name = self.entry_name.get().strip()
            new_email = self.entry_email.get().strip()
            if new_name and new_email:
                self.tree.item(selected_item, values=(new_name, new_email))
                del self.dealer_info[old_name]
                self.dealer_info[new_name] = new_email
                self.entry_name.delete(0, tk.END)
                self.entry_email.delete(0, tk.END)

    def delete_dealer(self):
        selected_item = self.tree.selection()
        if selected_item:
            item = self.tree.item(selected_item)
            name = item['values'][0]
            self.tree.delete(selected_item)
            del self.dealer_info[name]

    def save_and_close(self):
        with open('dealers.csv', 'w') as f:
            for name, email in self.dealer_info.items():
                f.write(f"{name},{email}\n")
        self.root.quit()

# 파일 경로 설정
base_path = r"D:\대리점자동화연습\4월 원본"
output_base_path = os.path.join(base_path, "4월 정산 전송")
os.makedirs(output_base_path, exist_ok=True)

summary_path = os.path.join(base_path, "2024.04월_파트너정산.cs")
new_detail_path = os.path.join(base_path, "2024.04월_파트너세부리스트.cs")

# 파일 인코딩 감지 함수
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# 파일 존재 여부 확인 및 인코딩 감지
def read_file_with_encoding(file_path, **kwargs):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_path} 파일이 존재하지 않습니다.")
    encoding = detect_encoding(file_path)
    return pd.read_csv(file_path, encoding=encoding, **kwargs)

def main():
    root = tk.Tk()
    app = EmailApp(root)
    root.mainloop()

    sender_email = simpledialog.askstring("Input", "이메일 주소를 입력하세요:")
    sender_password = simpledialog.askstring("Input", "비밀번호를 입력하세요:", show='*')

    df_summary = read_file_with_encoding(summary_path, header=6)
    df_new_detail = read_file_with_encoding(new_detail_path)
    df_cancel_detail = read_file_with_encoding(cancel_detail_path)

    df_summary = df_summary.loc[:, ~df_summary.columns.str.contains('Unnamed')]
    df_summary.columns = df_summary.columns.str.strip().str.replace('\n', '')

    required_columns = [
        '추천지사명', '상호명', '신규건수', '기계경비월정료', '판매수수료(서비스형)', '판매수수료(판매형)',
        '공급가액', 'VAT', '세후지급액', '최종지급액'
    ]

    if len(df_summary.columns) < len(required_columns):
        for col in required_columns:
            if col not in df_summary.columns:
                df_summary[col] = None

    df_summary = df_summary.fillna('')
    df_new_detail = df_new_detail.fillna('')
    df_cancel_detail = df_cancel_detail.fillna('')

    def fill_missing_values(df, target_row, rows_to_check, cols):
        for col in cols:
            for row in rows_to_check:
                if pd.isna(df.at[target_row, col]):
                    if not pd.isna(df.at[row, col]):
                        df.at[target_row, col] = df.at[row, col]
                        break

    columns_to_copy = df_summary.columns
    fill_missing_values(df_summary, 6, [5, 4], columns_to_copy)

    df_summary = df_summary[df_summary['본부'].str.contains("강북/강원본부", na=False)]
    df_summary = df_summary[~df_summary['본부'].str.contains('충남/충북본부|강남/서부본부|대구/경북본부|부산/경남본부|전남/전북본부', na=False)]
    df_summary = df_summary.drop(columns=['본부', '프로모션', '비고'], errors='ignore')

    total_row_index = df_summary[df_summary['상호명'].str.contains("총계", na=False)].index
    if not total_row_index.empty:
        df_summary = df_summary.drop(total_row_index)
        df_summary = df_summary.drop(total_row_index + 1, errors='ignore')

    df_new_detail['서비스번호'] = df_new_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_new_detail['고객번호'] = df_new_detail['고객번호'].astype(str).str.split('.').str[0]
    df_new_detail['계약번호'] = df_new_detail['계약번호'].astype(str).str.split('.').str[0]

    df_cancel_detail['서비스번호'] = df_cancel_detail['서비스번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['고객번호'] = df_cancel_detail['고객번호'].astype(str).str.split('.').str[0]
    df_cancel_detail['계약번호'] = df_cancel_detail['계약번호'].astype(str).str.split('.').str[0]
    agencies = ["에이치케이씨스템", "케이엔에스네트웍", "해빛씨엔에스", "비즈원플러스", "하슬라통신", "미라클 이노베이션", "성우네트웍스", "주식회사원정보통신"]

    dealer_info = {
            "에이치케이씨스템": "heebon.park@kt.com",
            "케이엔에스네트웍": "heebon.park@kt.com",
            "해빛씨엔에스": "bough38@naver.com",
            "비즈원플러스": "bough38@naver.com",
            "하슬라통신": "bough38@naver.com",
            "미라클 이노베이션": "bough38@naver.com",
            "성우네트웍스": "bough38@naver.com"
            "주식회사원정보통신": "bough38@naver.com", 
    }


    smtp_server = "smtp.naver.com"
    smtp_port = 587

    header_fill = PatternFill(start_color="4F4F4F", end_color="4F4F4F", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True)
    blue_font = Font(color="0000FF", bold=True)
    red_font = Font(color="FF0000", bold=True)
    thin_border = Border(left=Side(style='thin', color='000000'),
                         right=Side(style='thin', color='000000'),
                         top=Side(style='thin', color='000000'),
                         bottom=Side(style='thin', color='000000'))
    thick_border = Border(left=Side(style='thick', color='000000'),
                          right=Side(style='thick', color='000000'),
                          top=Side(style='thick', color='000000'),
                          bottom=Side(style='thick', color='000000'))

    for agency in agencies:
        summary_filtered = df_summary[df_summary['상호명'].str.contains(re.escape(agency), na=False)].copy()
       
        if summary_filtered.empty:
            print(f"{agency}에 대한 데이터가 없습니다.")
            continue

        summary_filtered = summary_filtered.drop_duplicates(subset=required_columns)
       
        new_detail_filtered = df_new_detail[df_new_detail['유통망대분류'].str.contains(re.escape(agency), na=False)].copy()
        new_detail_filtered = new_detail_filtered.drop(columns=['관리본부명', '서비스번호', '서비스(대)', '서비스(중)', '영업자사번', '추천본부명', '추천자사번', '추천자유형', '비고'], errors='ignore')

      
        with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
            summary_filtered.to_excel(writer, sheet_name="2024.04월_파트너정산", index=False)
            new_detail_filtered.to_excel(writer, sheet_name="2024.04월_파트너세부리스트", index=False)
      
        wb = openpyxl.load_workbook(excel_output_path)
       
        for sheet in wb.sheetnames:
            ws = wb[sheet]
           
            for cell in ws[1]:
                cell.fill = header_fill
                cell.font = header_font
                cell.border = thin_border
           
            if sheet == "2024.04월_파트너정산":
                ws.insert_rows(1)
                ws['L1'] = '최종지급액'
                ws['L1'].font = Font(bold=True)
               

            if sheet == "2024.04월_파트너세부리스트":
                ws.insert_rows(1)
                ws['N1'] = '월정료'
                ws['N1'].font = Font(bold=True)
                ws['D1'] = new_detail_filtered['계약번호'].nunique()
                ws['D1'].font = Font(bold=True)
                ws['D1'].number_format = '#,##0'
                ws['AD1'] = new_detail_filtered['환산월정료'].replace({',': ''}, regex=True).astype(float).sum()
                ws['AD1'].font = Font(bold=True, color='0000FF')
                ws['AD1'].number_format = '#,##0'

          
       
        wb.save(excel_output_path)

        receiver_email = dealer_info[agency]
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = receiver_email
        msg['Subject'] = f"4월 정산실적_{agency}_대리점_(표(요약). 신규상세, 해지상세)"

        summary_html = f"""
            <p><b>2024.04월_파트너정산:</b></p>
            <p>ㅇ 최종지급액: {summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</p>
            <p>ㅇ 최종지급액: <span style="color: blue; font-weight: bold;">{summary_filtered['최종지급액'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in summary_filtered.columns])}
                </tr>
                {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in summary_filtered.values])}
            </table>
        """

        new_detail_html = f"""
            <p><b>2024.04월_파트너세부리스트:</b> <span style="color: blue; font-weight: bold;">지급수수료 합계(환산월정료): {new_detail_filtered['지급수수료(환산월정료)'].replace({',': ''}, regex=True).astype(float).sum():,.0f}</span></p>
            <table border="1" cellspacing="0" cellpadding="5" style="font-size:12px;">
                <tr style="background-color: #4F4F4F; color: #FFFFFF; font-weight: bold;">
                    {''.join([f'<th>{col}</th>' for col in new_detail_filtered.columns])}
                </tr>
                  {''.join([f'<tr>{"".join([f"<td>{cell}</td>" for cell in row])}</tr>' for row in new_detail_filtered.values])}
            </table>
        """

   else:
            cancel_detail_html = ""

        body = f"""
        <!DOCTYPE html>
        <html>
        <body>
            <p style="font-size:12px;">{agency}님, 안녕하세요.</p>
            <p style="font-size:12px;">kt텔레캅 사외채널 담당자 장재혁입니다.</p>
            <p style="font-size:12px;">파트너 정산실적을 아래와 같이 송부드리니 6월 5일까지 세금계산서 발행 요청드리며 검토 후 특이사항 있으실 경우 연락주시기 바랍니다.</p>
            {summary_html}
            {new_detail_html}
            {cancel_detail_html}
        </body>
        </html>
        """

        msg.attach(MIMEText(body, 'html'))

        with open(excel_output_path, "rb") as attachment:
            part = MIMEApplication(attachment.read(), _subtype="octet-stream")
            part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(excel_output_path))
            msg.attach(part)

        # 이메일 전송 함수
        def send_email():
            try:
                with smtplib.SMTP(smtp_server, smtp_port) as server:
                    server.starttls()
                    server.login(sender_email, sender_password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
                messagebox.showinfo("성공", f"{agency}에 대한 이메일을 {receiver_email}로 전송했습니다.")
            except smtplib.SMTPAuthenticationError:
                messagebox.showerror("오류", "이메일 또는 비밀번호가 잘못되었습니다. 다시 시도해주세요.")
            except Exception as e:
                messagebox.showerror("오류", f"이메일 전송 중 오류가 발생했습니다: {str(e)}")

        send_email()

    print("모든 파일이 처리되고 이메일이 전송되었습니다.")

if __name__ == "__main__":
    main()